In [1]:
import numpy as np
import pandas as pd
import random
import hashlib as hl # hash library
import itertools

### RSA (Asymmetric):

In [22]:
# RSA algorithm (derived from lab10/11)
def are_relatively_prime(a, b):
    """Return ``True`` if ``a`` and ``b`` are two relatively prime numbers.

    Two numbers are relatively prime if they share no common factors,
    i.e. there is no integer (except 1) that divides both.
    """
    for n in range(2, min(a, b) + 1):
        if a % n == b % n == 0:
            return False
    return True

def generate_key_pair(p,q):
    # Calculate N = p*q and r = (p-1)(q-1)
    N = p*q 
    r = (p-1)*(q-1) 
    
    # choose a random number e such that e < N and e and R share no common factors.
    for cand in range(3, r, 2):
        if are_relatively_prime(cand, r):
            e = cand
            break
            
    # calculate number d, such that ed mod (r) = 1
    d = 0
    for cand in range(3, r, 2):
        if cand * e % r == 1:
            d = cand
            break     
    
    # return public key and private key pairs.
    public_key = (N,e)
    private_key = (N,d)
    return public_key, private_key

def encrypt(public_key, message):
    # encrypt the given message with the public key
    return pow(message,public_key[1]) % public_key[0]

def encryptString(public_key, message):
    # encrypt the given message with the public key
    n, key = public_key
    return [pow(ord(char),key,n) for char in message]

def decrypt(private_key, message):
    # retrieve the original message
    return pow(message,private_key[1]) % private_key[0]

def decryptString(private_key, message):
    # retrieve the original message from the encryoted message with you
    n, key = private_key
    return  ''.join([chr(pow(char, key, n)) for char in message])

In [23]:
# test cases
p = 5
q = 11
msg = random.randint(1,p*q)
public_key, private_key = generate_key_pair(5,11)
print ('public key is {}, private key is {}'.format(public_key, private_key) )
print ('message: {}'.format(msg))
print ('encrypted message: {}'.format(encrypt(public_key,msg)))
print ('decrypted message: {}'.format(decrypt(private_key,encrypt(public_key,msg) )))

public key is (55, 3), private key is (55, 27)
message: 32
encrypted message: 43
decrypted message: 32


### Diffie-Helman:

In [27]:
# derived from lab10
class Person:
    def __init__(self,p,g):
        self.p = p
        self.g = g
        self.my_secrete_number = random.randint(1,10)
        self.others_secrete_number = 0
        
    def tell(self, person):
        person.others_secrete_number = self.compute_number_to_share()
    
    def compute_number_to_share(self):
        return pow(self.g, self.my_secrete_number) % self.p
    
    def get_common_key(self):
        return pow(self.others_secrete_number, self.my_secrete_number) % self.p

In [28]:
prime_number = 23
generator = 11

Alice = Person(prime_number,generator)
Bob = Person(prime_number,generator)

Alice.tell(Bob)
Bob.tell(Alice)

print("Alice's key is: {}".format(Alice.get_common_key()))
print("Bob's key is: {}".format(Bob.get_common_key()))

Alice's key is: 19
Bob's key is: 19


### Hashing via hashlib

In [3]:
message = 'this is a short test string'
md5_hash = hl.md5(message.encode("UTF-8")).hexdigest()
sha_hash = hl.sha256(message.encode("UTF-8")).hexdigest()
blake2b = hl.blake2b(message.encode("UTF-8")).hexdigest()

print (f"md5: {md5_hash}")
print (f"sha: {sha_hash}")
print (f"blake2b: {blake2b}")

md5: 0af57bc31c06d32bb81e6c7171a1e51e
sha: db44d7adb45d2edaffd11d29d1f83a44308c26978a76f6d13ba99c28fbccfe53
blake2b: a13c9bff609acfac9225e0301b8dcbedd3715b29c6f5391a774dbda75997359f1ec45cda7fa63d1b88ab914bdc019717d39927d1f34cd2e17fa9fceee7e31759


### Determine Hash Properties:
- Collision Resistance - It should be computationally infeasible to find two different messages that hash to the same value. That is:
```
M = Message
M' = Another Message
M != M' (Messages are not the same)
H() = Hash Function 
H(M) != H(M')
```
If `H(M) == H(M')`, it is possible to falseify the hash. Obviously, not a good thing!
- Pre-image resistance - It should be computationally infeasible to determine the original message `M` given a hash function `H() `and a hashed message `H(M)`. Also known as _onewayness_
- 2nd-pre-image resistance - Given the knowledge of a hash function `H()` and a message `M`, it should be computationally infeasible to find another message `M'` that will hash to the same value.
- A small change in the original message should yield a large change in the resulting hash so it seems unrelated completely to a similar message.



In [9]:
# Your code here
# determine if a hash function is deterministic, collision resistant and pre-image resistant
# dpeth is max length of strings to check
def hashProperties(hashFunction, depth = 3):
    message1 = "this is a short test string"     
    message2 = "this is a short test strinh"
    message3 = "this is also a short test string"
    
    # check if deterministic   
    h1 = hashFunction(message1)
    h2 = hashFunction(message1)    
    print(f"Determinisitc: {h1 == h2}\n")
    
    # check if collision resistant
    allStrings = []
    for x in range(1, depth+1): # find all possible lowercase strings of length <= depth
        allStrings.extend(itertools.product([chr(c) for c in range(97,123)], repeat = x))
    allHashes = [hashFunction("".join(map(str,m))) for m in allStrings]    
    print(f"Collision resistant: {len(allHashes) == len(set(allHashes))}\n")
    
    # pre-image
        
    # length
    # check if hash length is equal between two different strings
    isLengthConst = len(hashFunction(message1)) == len(hashFunction(message3))
    # check hash length is equal for all possible lowercase strings of length <= depth
    for h in allHashes:
        if (len(h) != len(allHashes[0])):
            isLengthConst = False
            break
    print(f"Non-Variable Length: {isLengthConst}\n")
    
    
    # hashing different messages
    print("Minor change in M = major change in H(M):")
    print(f"'a' -> {hashFunction('a')}")
    print(f"'b' -> {hashFunction('b')}")
    print(f"'{message1}' -> {hashFunction(message1)}")
    print(f"'{message2}' -> {hashFunction(message2)}")
    print(f"'{message3}' -> {hashFunction(message3)}\n")

# convert a hashLib hash to a simple function that takes a message and returns the hash
def hashLibToFunction(hashLib):
    def returnFunction(message):
        hashFunction = hashLib.copy()
        hashFunction.update(message.encode("UTF-8"))
        return hashFunction.hexdigest()
    return returnFunction

# examples
print("md5:")
hashProperties(hashLibToFunction(hl.md5()))
print("sha256:")
hashProperties(hashLibToFunction(hl.sha256()))
print("blake2b:")
hashProperties(hashLibToFunction(hl.blake2b()))

md5:
Determinisitc: True

Collision resistant: True

Non-Variable Length: True

Minor change in M = major change in H(M):
'a' -> 0cc175b9c0f1b6a831c399e269772661
'b' -> 92eb5ffee6ae2fec3ad71c777531578f
'this is a short test string' -> 0af57bc31c06d32bb81e6c7171a1e51e
'this is a short test strinh' -> 895fd8c671a36c461a8d05083c7d59c8
'this is also a short test string' -> 5029f2d358cbc7f2416cffb5dacf217f

sha256:
Determinisitc: True

Collision resistant: True

Non-Variable Length: True

Minor change in M = major change in H(M):
'a' -> ca978112ca1bbdcafac231b39a23dc4da786eff8147c4e72b9807785afee48bb
'b' -> 3e23e8160039594a33894f6564e1b1348bbd7a0088d42c4acb73eeaed59c009d
'this is a short test string' -> db44d7adb45d2edaffd11d29d1f83a44308c26978a76f6d13ba99c28fbccfe53
'this is a short test strinh' -> 6df7736334004230781da87f913ba89a0d91d72890e4b85ed332acbb567714e4
'this is also a short test string' -> b884f93484d5716b9b5d40489858dabacac3dcb3b7e84e376cad72156bb18b43

blake2b:
Determinisitc: T

### Digital Signatures:

In [25]:
# gets primes in range (derived from lab11)
def get_primes(start, stop):
    """Return a list of prime numbers in ``range(start, stop)``."""
    if start >= stop:
        return []
    primes = [2]
    for n in range(3, stop + 1, 2):
        for p in primes:
            if n % p == 0:
                break
        else:
            primes.append(n)
    while primes and primes[0] < start:
        del primes[0]
    return primes

message = "super secret test message"

# generate keys
primes_candidates = get_primes(1000,9999)
p = random.choice(primes_candidates)
q = random.choice(primes_candidates)
public_key, private_key = generate_key_pair(p,q)

# compute hash
def hashFunction(message):
    hashed = hl.sha256(message.encode("UTF-8")).hexdigest()
    return hashed
hashedMessage = hashFunction(message)

# sign hash with private key
signature = encryptString(private_key, hashedMessage)

# signature verification
result = decryptString(public_key, signature)
print(hashedMessage == result)

True
